In [1]:
from stottrs import Mapping, to_graph
import polars as pl
from math import floor

In [2]:
pl.Config.set_fmt_str_lengths(300)

polars.cfg.Config

# Templates

In [3]:
instance_mapping = """
@prefix tpl:<https://github.com/magbak/chrontext/templates#>.
@prefix rds:<https://github.com/magbak/chrontext/rds_power#>.
@prefix ct:<https://github.com/magbak/chrontext#>.
@prefix rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs:<http://www.w3.org/2000/01/rdf-schema#>.

tpl:Site [?SiteIRI, ?SiteName] :: {
    ottr:Triple(?SiteIRI, rdfs:label, ?SiteName),
    ottr:Triple(?SiteIRI, rdf:type, rds:Site)
    } .
    
tpl:FunctionalAspect [?SourceIRI, ?TargetIRI, ?TargetAspectNodeIRI, ?Label] :: {
    ottr:Triple(?SourceIRI, rds:hasFunctionalAspect, ?TargetAspectNodeIRI),
    ottr:Triple(?TargetIRI, rds:hasFunctionalAspectNode, ?TargetAspectNodeIRI),
    ottr:Triple(?TargetAspectNodeIRI, rdfs:label, ?Label)
} .

tpl:RDSSystem [?SystemIRI, xsd:anyURI ?RDSType, ?Label] :: {
    ottr:Triple(?SystemIRI, rdf:type, ?RDSType),
    ottr:Triple(?SystemIRI, rdfs:label, ?Label)
} .

tpl:StaticProperty [?ParentIRI, ?ValueNodeIRI, ?Label, ?Value] :: {
    ottr:Triple(?ParentIRI, ct:hasStaticProperty, ?ValueNodeIRI),
    ottr:Triple(?ValueNodeIRI, rdfs:label, ?Label),
    ottr:Triple(?ValueNodeIRI, ct:hasStaticValue, ?Value)
} .

tpl:Timeseries [?ParentIRI, ?TimeseriesNodeIRI, ?Label, ?ExternalId, xsd:anyURI ?Datatype] :: {
    ottr:Triple(?ParentIRI, ct:hasTimeseries, ?TimeseriesNodeIRI),
    ottr:Triple(?TimeseriesNodeIRI, ct:hasExternalId, ?ExternalId),
    ottr:Triple(?TimeseriesNodeIRI, ct:hasDatatype, ?Datatype),
    ottr:Triple(?TimeseriesNodeIRI, rdfs:label, ?Label)
} .
"""

In [4]:
n = 40

# Mapping object

In [5]:
mapping = Mapping([instance_mapping])

In [6]:
# Used as a prefix
wpex = "https://github.com/magbak/chrontext/windpower_example#"
rds = "https://github.com/magbak/chrontext/rds_power#"

In [7]:
site_iris = [wpex + "Site" + str(i) for i in range(4)]
sites = pl.DataFrame({"SiteName": ["Wind Mountain", "Gale Valley", "Gusty Plains", "Breezy Field"],
                      "SiteIRI": site_iris})
sites

SiteName,SiteIRI
str,str
"""Wind Mountain""","""https://github.com/magbak/chrontext/windpower_example#Site0"""
"""Gale Valley""","""https://github.com/magbak/chrontext/windpower_example#Site1"""
"""Gusty Plains""","""https://github.com/magbak/chrontext/windpower_example#Site2"""
"""Breezy Field""","""https://github.com/magbak/chrontext/windpower_example#Site3"""


In [8]:
mapping.expand("tpl:Site", sites)

# Wind turbines

In [9]:
wind_turbine_iris = [wpex + "WindTurbine" + str(i) for i in range (1, n+1)]
wind_turbines = pl.DataFrame(
    {
    "RDSType": [rds + "A"]*n,
    "Label": ["Wind turbine " + str(i) for i in range(1,n+1)],
    "SystemIRI": wind_turbine_iris
    })
wind_turbines

RDSType,Label,SystemIRI
str,str,str
"""https://github.com/magbak/chrontext/rds_power#A""","""Wind turbine 1""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine1"""
"""https://github.com/magbak/chrontext/rds_power#A""","""Wind turbine 2""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine2"""
"""https://github.com/magbak/chrontext/rds_power#A""","""Wind turbine 3""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine3"""
"""https://github.com/magbak/chrontext/rds_power#A""","""Wind turbine 4""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine4"""
"""https://github.com/magbak/chrontext/rds_power#A""","""Wind turbine 5""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine5"""
"""https://github.com/magbak/chrontext/rds_power#A""","""Wind turbine 6""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine6"""
"""https://github.com/magbak/chrontext/rds_power#A""","""Wind turbine 7""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine7"""
"""https://github.com/magbak/chrontext/rds_power#A""","""Wind turbine 8""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine8"""
"""https://github.com/magbak/chrontext/rds_power#A""","""Wind turbine 9""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine9"""


In [10]:
mapping.expand("tpl:RDSSystem", wind_turbines)

In [11]:
operating_external_ids = ["oper" + str(i) for i in range(1,n+1)]
operating = pl.DataFrame ({
    "Label": ["Operating"]*n,
    "ParentIRI" :wind_turbine_iris,
    "ExternalId": operating_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#boolean"]*n
})
operating = operating.with_column(pl.Series("TimeseriesNodeIRI", [wpex]*operating.height) + operating["ExternalId"])
operating

Label,ParentIRI,ExternalId,Datatype,TimeseriesNodeIRI
str,str,str,str,str
"""Operating""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine1""","""oper1""","""http://www.w3.org/2001/XMLSchema#boolean""","""https://github.com/magbak/chrontext/windpower_example#oper1"""
"""Operating""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine2""","""oper2""","""http://www.w3.org/2001/XMLSchema#boolean""","""https://github.com/magbak/chrontext/windpower_example#oper2"""
"""Operating""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine3""","""oper3""","""http://www.w3.org/2001/XMLSchema#boolean""","""https://github.com/magbak/chrontext/windpower_example#oper3"""
"""Operating""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine4""","""oper4""","""http://www.w3.org/2001/XMLSchema#boolean""","""https://github.com/magbak/chrontext/windpower_example#oper4"""
"""Operating""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine5""","""oper5""","""http://www.w3.org/2001/XMLSchema#boolean""","""https://github.com/magbak/chrontext/windpower_example#oper5"""
"""Operating""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine6""","""oper6""","""http://www.w3.org/2001/XMLSchema#boolean""","""https://github.com/magbak/chrontext/windpower_example#oper6"""
"""Operating""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine7""","""oper7""","""http://www.w3.org/2001/XMLSchema#boolean""","""https://github.com/magbak/chrontext/windpower_example#oper7"""
"""Operating""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine8""","""oper8""","""http://www.w3.org/2001/XMLSchema#boolean""","""https://github.com/magbak/chrontext/windpower_example#oper8"""
"""Operating""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine9""","""oper9""","""http://www.w3.org/2001/XMLSchema#boolean""","""https://github.com/magbak/chrontext/windpower_example#oper9"""


In [12]:
mapping.expand("tpl:Timeseries", operating)

In [13]:
maximum_power_values = [[5_000_000,10_000_000,15_000_000][i%3] for i in range(1,n+1)]
maximum_power_node_iri = [wpex + "WindTurbineMaximumPower" + str(i) for i in range(1, n+1)]
maximum_power = pl.DataFrame ({
    "Label": ["MaximumPower"] * n,
    "Value": maximum_power_values,
    "ParentIRI": wind_turbine_iris,
    "ValueNodeIRI": maximum_power_node_iri
})
maximum_power

Label,Value,ParentIRI,ValueNodeIRI
str,i64,str,str
"""MaximumPower""",10000000,"""https://github.com/magbak/chrontext/windpower_example#WindTurbine1""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineMaximumPower1"""
"""MaximumPower""",15000000,"""https://github.com/magbak/chrontext/windpower_example#WindTurbine2""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineMaximumPower2"""
"""MaximumPower""",5000000,"""https://github.com/magbak/chrontext/windpower_example#WindTurbine3""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineMaximumPower3"""
"""MaximumPower""",10000000,"""https://github.com/magbak/chrontext/windpower_example#WindTurbine4""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineMaximumPower4"""
"""MaximumPower""",15000000,"""https://github.com/magbak/chrontext/windpower_example#WindTurbine5""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineMaximumPower5"""
"""MaximumPower""",5000000,"""https://github.com/magbak/chrontext/windpower_example#WindTurbine6""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineMaximumPower6"""
"""MaximumPower""",10000000,"""https://github.com/magbak/chrontext/windpower_example#WindTurbine7""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineMaximumPower7"""
"""MaximumPower""",15000000,"""https://github.com/magbak/chrontext/windpower_example#WindTurbine8""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineMaximumPower8"""
"""MaximumPower""",5000000,"""https://github.com/magbak/chrontext/windpower_example#WindTurbine9""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineMaximumPower9"""


In [14]:
mapping.expand("tpl:StaticProperty", 
               df=maximum_power)

In [15]:
def add_aspect_labeling_by_source(df:pl.DataFrame, prefix:str) -> pl.DataFrame:
   label_df = df.groupby("SourceIRI", maintain_order=True).apply(lambda x:pl.DataFrame({"Label":[prefix + str(i) for i in range(1,x.height+1)]}))
   df = df.with_column(label_df["Label"])
   return df

In [16]:
target_aspect_node_iri = [wpex + "WindTurbineFunctionalAspect" + str(i) for i in range(1, n+1)]

site_has_wind_turbine = pl.DataFrame({
    "SourceIRI":[site_iris[floor(i/(n/sites.height))] for i in range(0,n)],
    "TargetIRI":wind_turbine_iris,
    "TargetAspectNodeIRI": target_aspect_node_iri
})
site_has_wind_turbine = add_aspect_labeling_by_source(site_has_wind_turbine, "A")
site_has_wind_turbine

SourceIRI,TargetIRI,TargetAspectNodeIRI,Label
str,str,str,str
"""https://github.com/magbak/chrontext/windpower_example#Site0""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine1""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineFunctionalAspect1""","""A1"""
"""https://github.com/magbak/chrontext/windpower_example#Site0""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine2""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineFunctionalAspect2""","""A2"""
"""https://github.com/magbak/chrontext/windpower_example#Site0""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine3""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineFunctionalAspect3""","""A3"""
"""https://github.com/magbak/chrontext/windpower_example#Site0""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine4""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineFunctionalAspect4""","""A4"""
"""https://github.com/magbak/chrontext/windpower_example#Site0""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine5""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineFunctionalAspect5""","""A5"""
"""https://github.com/magbak/chrontext/windpower_example#Site0""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine6""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineFunctionalAspect6""","""A6"""
"""https://github.com/magbak/chrontext/windpower_example#Site0""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine7""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineFunctionalAspect7""","""A7"""
"""https://github.com/magbak/chrontext/windpower_example#Site0""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine8""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineFunctionalAspect8""","""A8"""
"""https://github.com/magbak/chrontext/windpower_example#Site0""","""https://github.com/magbak/chrontext/windpower_example#WindTurbine9""","""https://github.com/magbak/chrontext/windpower_example#WindTurbineFunctionalAspect9""","""A9"""


In [17]:
mapping.expand("tpl:FunctionalAspect", site_has_wind_turbine)

# Generator systems

In [18]:
generator_system_iris = [wpex + "GeneratorSystem" + str(i) for i in range(1,n+1)]
generator_systems = pl.DataFrame(
    {
    "SystemIRI":generator_system_iris,
    "Label": ["Generator system"]*n,
    "RDSType": [rds + "RA"]*n
    })
generator_systems

SystemIRI,Label,RDSType
str,str,str
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem1""","""Generator system""","""https://github.com/magbak/chrontext/rds_power#RA"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem2""","""Generator system""","""https://github.com/magbak/chrontext/rds_power#RA"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem3""","""Generator system""","""https://github.com/magbak/chrontext/rds_power#RA"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem4""","""Generator system""","""https://github.com/magbak/chrontext/rds_power#RA"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem5""","""Generator system""","""https://github.com/magbak/chrontext/rds_power#RA"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem6""","""Generator system""","""https://github.com/magbak/chrontext/rds_power#RA"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem7""","""Generator system""","""https://github.com/magbak/chrontext/rds_power#RA"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem8""","""Generator system""","""https://github.com/magbak/chrontext/rds_power#RA"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem9""","""Generator system""","""https://github.com/magbak/chrontext/rds_power#RA"""


In [19]:
mapping.expand("tpl:RDSSystem", generator_systems)

In [20]:
wind_turbine_has_generator_system = pl.DataFrame ({
    "SourceIRI": wind_turbine_iris,
    "TargetIRI": generator_system_iris,
    "TargetAspectNodeIRI": [wpex + "GeneratorSystemFunctionalAspect" + str(i) for i in range(1, n+1)]
})
wind_turbine_has_generator_system = add_aspect_labeling_by_source(wind_turbine_has_generator_system, "RA")
wind_turbine_has_generator_system

SourceIRI,TargetIRI,TargetAspectNodeIRI,Label
str,str,str,str
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine1""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem1""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystemFunctionalAspect1""","""RA1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine2""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem2""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystemFunctionalAspect2""","""RA1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine3""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem3""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystemFunctionalAspect3""","""RA1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine4""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem4""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystemFunctionalAspect4""","""RA1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine5""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem5""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystemFunctionalAspect5""","""RA1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine6""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem6""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystemFunctionalAspect6""","""RA1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine7""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem7""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystemFunctionalAspect7""","""RA1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine8""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem8""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystemFunctionalAspect8""","""RA1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine9""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem9""","""https://github.com/magbak/chrontext/windpower_example#GeneratorSystemFunctionalAspect9""","""RA1"""


In [21]:
mapping.expand("tpl:FunctionalAspect", wind_turbine_has_generator_system)

# Generator

In [22]:
generator_iris = [wpex + "Generator" + str(i) for i in range(1,n+1)]
generators = pl.DataFrame(
    {
    "SystemIRI": generator_iris,
    "RDSType": [rds + "GAA"]*n,
    "Label" : ["Generator"]*n
    })
generators

SystemIRI,RDSType,Label
str,str,str
"""https://github.com/magbak/chrontext/windpower_example#Generator1""","""https://github.com/magbak/chrontext/rds_power#GAA""","""Generator"""
"""https://github.com/magbak/chrontext/windpower_example#Generator2""","""https://github.com/magbak/chrontext/rds_power#GAA""","""Generator"""
"""https://github.com/magbak/chrontext/windpower_example#Generator3""","""https://github.com/magbak/chrontext/rds_power#GAA""","""Generator"""
"""https://github.com/magbak/chrontext/windpower_example#Generator4""","""https://github.com/magbak/chrontext/rds_power#GAA""","""Generator"""
"""https://github.com/magbak/chrontext/windpower_example#Generator5""","""https://github.com/magbak/chrontext/rds_power#GAA""","""Generator"""
"""https://github.com/magbak/chrontext/windpower_example#Generator6""","""https://github.com/magbak/chrontext/rds_power#GAA""","""Generator"""
"""https://github.com/magbak/chrontext/windpower_example#Generator7""","""https://github.com/magbak/chrontext/rds_power#GAA""","""Generator"""
"""https://github.com/magbak/chrontext/windpower_example#Generator8""","""https://github.com/magbak/chrontext/rds_power#GAA""","""Generator"""
"""https://github.com/magbak/chrontext/windpower_example#Generator9""","""https://github.com/magbak/chrontext/rds_power#GAA""","""Generator"""


In [23]:
mapping.expand("tpl:RDSSystem", generators)

In [24]:
energy_production_external_ids = ["ep" + str(i) for i in range(1,n+1)]
energy_production = pl.DataFrame ({
    "Label": ["Production"]*n,
    "ParentIRI" :generator_iris,
    "ExternalId": energy_production_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#double"]*n
})
energy_production = energy_production.with_column(pl.Series("TimeseriesNodeIRI", [wpex]*energy_production.height)  + energy_production["ExternalId"])
energy_production

Label,ParentIRI,ExternalId,Datatype,TimeseriesNodeIRI
str,str,str,str,str
"""Production""","""https://github.com/magbak/chrontext/windpower_example#Generator1""","""ep1""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#ep1"""
"""Production""","""https://github.com/magbak/chrontext/windpower_example#Generator2""","""ep2""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#ep2"""
"""Production""","""https://github.com/magbak/chrontext/windpower_example#Generator3""","""ep3""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#ep3"""
"""Production""","""https://github.com/magbak/chrontext/windpower_example#Generator4""","""ep4""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#ep4"""
"""Production""","""https://github.com/magbak/chrontext/windpower_example#Generator5""","""ep5""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#ep5"""
"""Production""","""https://github.com/magbak/chrontext/windpower_example#Generator6""","""ep6""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#ep6"""
"""Production""","""https://github.com/magbak/chrontext/windpower_example#Generator7""","""ep7""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#ep7"""
"""Production""","""https://github.com/magbak/chrontext/windpower_example#Generator8""","""ep8""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#ep8"""
"""Production""","""https://github.com/magbak/chrontext/windpower_example#Generator9""","""ep9""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#ep9"""


In [25]:
mapping.expand("tpl:Timeseries", energy_production)

In [26]:
generator_system_has_generator = pl.DataFrame ({
    "SourceIRI": generator_system_iris,
    "TargetIRI": generator_iris,
    "TargetAspectNodeIRI": [wpex + "GeneratorFunctionalAspect" + str(i) for i in range(1, n+1)]
})
generator_system_has_generator = add_aspect_labeling_by_source(generator_system_has_generator, "GAA")
generator_system_has_generator

SourceIRI,TargetIRI,TargetAspectNodeIRI,Label
str,str,str,str
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem1""","""https://github.com/magbak/chrontext/windpower_example#Generator1""","""https://github.com/magbak/chrontext/windpower_example#GeneratorFunctionalAspect1""","""GAA1"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem2""","""https://github.com/magbak/chrontext/windpower_example#Generator2""","""https://github.com/magbak/chrontext/windpower_example#GeneratorFunctionalAspect2""","""GAA1"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem3""","""https://github.com/magbak/chrontext/windpower_example#Generator3""","""https://github.com/magbak/chrontext/windpower_example#GeneratorFunctionalAspect3""","""GAA1"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem4""","""https://github.com/magbak/chrontext/windpower_example#Generator4""","""https://github.com/magbak/chrontext/windpower_example#GeneratorFunctionalAspect4""","""GAA1"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem5""","""https://github.com/magbak/chrontext/windpower_example#Generator5""","""https://github.com/magbak/chrontext/windpower_example#GeneratorFunctionalAspect5""","""GAA1"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem6""","""https://github.com/magbak/chrontext/windpower_example#Generator6""","""https://github.com/magbak/chrontext/windpower_example#GeneratorFunctionalAspect6""","""GAA1"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem7""","""https://github.com/magbak/chrontext/windpower_example#Generator7""","""https://github.com/magbak/chrontext/windpower_example#GeneratorFunctionalAspect7""","""GAA1"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem8""","""https://github.com/magbak/chrontext/windpower_example#Generator8""","""https://github.com/magbak/chrontext/windpower_example#GeneratorFunctionalAspect8""","""GAA1"""
"""https://github.com/magbak/chrontext/windpower_example#GeneratorSystem9""","""https://github.com/magbak/chrontext/windpower_example#Generator9""","""https://github.com/magbak/chrontext/windpower_example#GeneratorFunctionalAspect9""","""GAA1"""


In [27]:
mapping.expand("tpl:FunctionalAspect", generator_system_has_generator)

# Weather measuring system

In [28]:
wms_iris = [wpex + "WeatherMeasuringSystem" + str(i) for i in range(1,n+1)]
wms = pl.DataFrame(
    {
    "SystemIRI":wms_iris,
    "RDSType": [rds + "LE"]*n,
    "Label": ["Weather Measuring System"]*n
    })
wms

SystemIRI,RDSType,Label
str,str,str
"""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem1""","""https://github.com/magbak/chrontext/rds_power#LE""","""Weather Measuring System"""
"""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem2""","""https://github.com/magbak/chrontext/rds_power#LE""","""Weather Measuring System"""
"""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem3""","""https://github.com/magbak/chrontext/rds_power#LE""","""Weather Measuring System"""
"""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem4""","""https://github.com/magbak/chrontext/rds_power#LE""","""Weather Measuring System"""
"""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem5""","""https://github.com/magbak/chrontext/rds_power#LE""","""Weather Measuring System"""
"""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem6""","""https://github.com/magbak/chrontext/rds_power#LE""","""Weather Measuring System"""
"""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem7""","""https://github.com/magbak/chrontext/rds_power#LE""","""Weather Measuring System"""
"""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem8""","""https://github.com/magbak/chrontext/rds_power#LE""","""Weather Measuring System"""
"""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem9""","""https://github.com/magbak/chrontext/rds_power#LE""","""Weather Measuring System"""


In [29]:
mapping.expand("tpl:RDSSystem", wms)

In [30]:
wind_speed_external_ids = ["wsp" + str(i) for i in range(1,n+1)]
wind_speed = pl.DataFrame ({
    "Label": ["Windspeed"]*n,
    "ParentIRI" :wms_iris,
    "ExternalId": wind_speed_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#double"]*n
})
wind_speed = wind_speed.with_column(pl.Series("TimeseriesNodeIRI", [wpex]*wind_speed.height)  + wind_speed["ExternalId"])
wind_speed

Label,ParentIRI,ExternalId,Datatype,TimeseriesNodeIRI
str,str,str,str,str
"""Windspeed""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem1""","""wsp1""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wsp1"""
"""Windspeed""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem2""","""wsp2""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wsp2"""
"""Windspeed""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem3""","""wsp3""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wsp3"""
"""Windspeed""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem4""","""wsp4""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wsp4"""
"""Windspeed""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem5""","""wsp5""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wsp5"""
"""Windspeed""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem6""","""wsp6""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wsp6"""
"""Windspeed""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem7""","""wsp7""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wsp7"""
"""Windspeed""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem8""","""wsp8""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wsp8"""
"""Windspeed""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem9""","""wsp9""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wsp9"""


In [31]:
mapping.expand("tpl:Timeseries", wind_speed)

In [32]:
wind_direction_external_ids = ["wdir" + str(i) for i in range(1,n+1)]
wind_direction = pl.DataFrame ({
    "Label": ["WindDirection"]*n,
    "ParentIRI" :wms_iris,
    "ExternalId": wind_direction_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#double"]*n
})
wind_direction = wind_direction.with_column(pl.Series("TimeseriesNodeIRI", [wpex]*wind_direction.height)  + wind_direction["ExternalId"])
wind_direction

Label,ParentIRI,ExternalId,Datatype,TimeseriesNodeIRI
str,str,str,str,str
"""WindDirection""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem1""","""wdir1""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wdir1"""
"""WindDirection""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem2""","""wdir2""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wdir2"""
"""WindDirection""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem3""","""wdir3""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wdir3"""
"""WindDirection""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem4""","""wdir4""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wdir4"""
"""WindDirection""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem5""","""wdir5""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wdir5"""
"""WindDirection""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem6""","""wdir6""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wdir6"""
"""WindDirection""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem7""","""wdir7""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wdir7"""
"""WindDirection""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem8""","""wdir8""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wdir8"""
"""WindDirection""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem9""","""wdir9""","""http://www.w3.org/2001/XMLSchema#double""","""https://github.com/magbak/chrontext/windpower_example#wdir9"""


In [33]:
mapping.expand("tpl:Timeseries", wind_direction)

In [34]:
wind_turbine_has_wms = pl.DataFrame ({
    "SourceIRI": wind_turbine_iris,
    "TargetIRI": wms_iris,
    "TargetAspectNodeIRI": [wpex + "WMSFunctionalAspect" + str(i) for i in range(1, n+1)]
})
wind_turbine_has_wms = add_aspect_labeling_by_source(wind_turbine_has_wms, "LE")
wind_turbine_has_wms

SourceIRI,TargetIRI,TargetAspectNodeIRI,Label
str,str,str,str
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine1""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem1""","""https://github.com/magbak/chrontext/windpower_example#WMSFunctionalAspect1""","""LE1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine2""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem2""","""https://github.com/magbak/chrontext/windpower_example#WMSFunctionalAspect2""","""LE1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine3""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem3""","""https://github.com/magbak/chrontext/windpower_example#WMSFunctionalAspect3""","""LE1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine4""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem4""","""https://github.com/magbak/chrontext/windpower_example#WMSFunctionalAspect4""","""LE1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine5""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem5""","""https://github.com/magbak/chrontext/windpower_example#WMSFunctionalAspect5""","""LE1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine6""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem6""","""https://github.com/magbak/chrontext/windpower_example#WMSFunctionalAspect6""","""LE1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine7""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem7""","""https://github.com/magbak/chrontext/windpower_example#WMSFunctionalAspect7""","""LE1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine8""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem8""","""https://github.com/magbak/chrontext/windpower_example#WMSFunctionalAspect8""","""LE1"""
"""https://github.com/magbak/chrontext/windpower_example#WindTurbine9""","""https://github.com/magbak/chrontext/windpower_example#WeatherMeasuringSystem9""","""https://github.com/magbak/chrontext/windpower_example#WMSFunctionalAspect9""","""LE1"""


In [35]:
mapping.expand("tpl:FunctionalAspect", 
               df=wind_turbine_has_wms)

In [36]:
mapping.write_ntriples("windpower.nt")

In [37]:
df_external_ids = pl.DataFrame({
    "MaximumPowerValue": maximum_power_values,
    "Operating":operating_external_ids,
    "EnergyProduction":energy_production_external_ids,
    "WindSpeed":wind_speed_external_ids,
    "WindDirection":wind_direction_external_ids
})
df_external_ids.write_csv("external_ids.csv")

In [38]:
gr = to_graph(mapping)

In [39]:
gr

<Graph identifier=N10e11b4a76a44de2a7fc2822ab769219 (<class 'rdflib.graph.Graph'>)>

In [40]:
res = gr.query("SELECT * WHERE {?a a ?b}")

In [41]:
for r in res:
    print(r)

(rdflib.term.URIRef('https://github.com/magbak/chrontext/windpower_example#Site0'), rdflib.term.URIRef('https://github.com/magbak/chrontext/rds_power#Site'))
(rdflib.term.URIRef('https://github.com/magbak/chrontext/windpower_example#Site1'), rdflib.term.URIRef('https://github.com/magbak/chrontext/rds_power#Site'))
(rdflib.term.URIRef('https://github.com/magbak/chrontext/windpower_example#Site2'), rdflib.term.URIRef('https://github.com/magbak/chrontext/rds_power#Site'))
(rdflib.term.URIRef('https://github.com/magbak/chrontext/windpower_example#Site3'), rdflib.term.URIRef('https://github.com/magbak/chrontext/rds_power#Site'))
(rdflib.term.URIRef('https://github.com/magbak/chrontext/windpower_example#WindTurbine1'), rdflib.term.URIRef('https://github.com/magbak/chrontext/rds_power#A'))
(rdflib.term.URIRef('https://github.com/magbak/chrontext/windpower_example#WindTurbine2'), rdflib.term.URIRef('https://github.com/magbak/chrontext/rds_power#A'))
(rdflib.term.URIRef('https://github.com/magb

In [42]:
res2 = gr.query("""PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
PREFIX ct:<https://github.com/magbak/chrontext#>
PREFIX wp:<https://github.com/magbak/chrontext/windpower_example#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rds:<https://github.com/magbak/chrontext/rds_power#>
SELECT ?site_label ?wtur_label ?ts ?ts_label WHERE {
    ?site a rds:Site .
    ?site rdfs:label ?site_label .
    ?site rds:hasFunctionalAspect ?wtur_asp .
    ?wtur_asp rdfs:label ?wtur_label .
    ?wtur rds:hasFunctionalAspectNode ?wtur_asp .
    ?wtur rds:hasFunctionalAspect ?gensys_asp .
    ?wtur a rds:A .
    ?gensys rds:hasFunctionalAspectNode ?gensys_asp .
    ?gensys a rds:RA .
    ?gensys rds:hasFunctionalAspect ?generator_asp .
    ?generator rds:hasFunctionalAspectNode ?generator_asp .
    ?generator a rds:GAA .
    ?generator ct:hasTimeseries ?ts .
    ?ts rdfs:label ?ts_label .
    FILTER(?wtur_label = "A1" && ?site_label = "Wind Mountain") .
}""")

In [43]:
for r in res2:
    print(r)

(rdflib.term.Literal('Wind Mountain', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('A1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.URIRef('https://github.com/magbak/chrontext/windpower_example#ep1'), rdflib.term.Literal('Production', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))


KeyboardInterrupt: 

In [ ]:
res3 = gr.query("""PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
PREFIX ct:<https://github.com/magbak/chrontext#>
PREFIX wp:<https://github.com/magbak/chrontext/windpower_example#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rds:<https://github.com/magbak/chrontext/rds_power#>
SELECT ?site_label ?wtur_label ?ts_speed ?ts_dir ?ts_prod WHERE {
    ?site a rds:Site .
    ?site rdfs:label ?site_label .
    ?site rds:hasFunctionalAspect ?wtur_asp .
    ?wtur_asp rdfs:label ?wtur_label .
    ?wtur rds:hasFunctionalAspectNode ?wtur_asp .
    ?wtur a rds:A .
    ?wtur rds:hasFunctionalAspect ?gensys_asp .
    ?gensys rds:hasFunctionalAspectNode ?gensys_asp .
    ?gensys a rds:RA .
    ?gensys rds:hasFunctionalAspect ?generator_asp .
    ?generator rds:hasFunctionalAspectNode ?generator_asp .
    ?generator a rds:GAA .
    ?wtur rds:hasFunctionalAspect ?weather_asp .
    ?weather rds:hasFunctionalAspectNode ?weather_asp .
    ?weather a rds:LE .
    ?weather ct:hasTimeseries ?ts_speed .
    ?ts_speed rdfs:label "Windspeed" .
    ?weather ct:hasTimeseries ?ts_dir .
    ?ts_dir rdfs:label "WindDirection" .
    ?generator ct:hasTimeseries ?ts_prod .
    ?ts_prod rdfs:label "Production" .
    FILTER(?site_label = "Wind Mountain"
    && ?wtur_label = "A1") .
}
""")

In [ ]:
for r in res3:
    print(r)